In [1]:
# importing all necessary libs
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import dash
import os
import dash_core_components as dcc
import dash_html_components as html
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)

In [2]:
# Data reading
df_death_injury = pd.read_excel(os.path.abspath(parent_dir+'/data/xlsx/'+'death_injury_yearly.xlsx'))
df_attacks_targets = pd.read_excel(os.path.abspath(parent_dir+'/data/xlsx/'+'attacks_by_target.xlsx'))
df_groups = pd.read_excel(os.path.abspath(parent_dir+'/data/xlsx/'+'group_year_dataset.xlsx'))

with open(os.path.abspath(parent_dir+'/data/'+'data_description.txt')) as f:
    desc = f.readlines()
f.close()

In [3]:
df_attacks_targets.sort_values(by='eventid')

,Unnamed: 0,targtype1_txt,iyear,country_txt,id,eventid,nkill,nwound
0,0,Abortion Related,2000,Canada,CAN,1,0,1
3824,3824,NGO,2018,Ukraine,UKR,1,0,1
3827,3827,NGO,2019,Burkina Faso,BFA,1,0,0
3828,3828,NGO,2019,Cameroon,CMR,1,1,0
3829,3829,NGO,2019,Central African Republic,CAF,1,1,0
...,...,...,...,...,...,...,...,...
5191,5191,Private Citizens & Property,2013,Iraq,IRQ,873,2610,7562
5446,5446,Private Citizens & Property,2017,Iraq,IRQ,932,2792,2608
5312,5312,Private Citizens & Property,2015,Iraq,IRQ,1056,3373,6328
5247,5247,Private Citizens & Property,2014,Iraq,IRQ,1306,5323,7655


In [23]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Global Terrorism Dashboard'


def show_world_map():
    fig = px.scatter_geo(df_death_injury, 
                     locations="id",
                     hover_name="country_txt", 
                     size="eventid",
                     animation_frame="iyear",
                     height=800,
                     projection="natural earth",
                     color_discrete_sequence=[ 'red', ],
                     template="simple_white")
    
    
    
    return fig

def show_first_five_groups():
    first_five= df_groups.groupby(by='gname').agg({'nkill':'sum'}).sort_values(by = 'nkill' ,ascending=False).reset_index().head(5)['gname']
    df = df_groups.loc[df_groups['gname'].isin(first_five)]
    fig = px.area(df, 
                  x="iyear", 
                  y="nkill", 
                  color="gname",
                  line_group="country_txt",
                  template="simple_white")
    fig.update_layout(
                      #plot_bgcolor="white",
                      xaxis_title="Years",
                      yaxis_title="Number of Attacks",
                      legend_title="Group Names")

    return fig


app.layout = html.Div([
    html.Div([html.H1("Global Terrorism Dashboard")],
                 style={'textAlign': "center", "padding-bottom": "30"}, 
                 className="six-column"),
    html.Div([html.P(dcc.Markdown(desc))],
                 style={'textAlign': "left", "padding-bottom": "30"}, 
                 className="six-column"),
    html.Div([html.H3('The Presentation of Yearly Terrorist Attacks',
                 style={'textAlign': "center", "padding-bottom": "30"}),
              dcc.Graph(id="world-map", 
                       style = {'textAlign': "center"}, 
                       figure = show_world_map(), 
                       className="twelve columns"),]), 
    html.Div([html.H3('The Most Dangerous Terrorist Groups over The Years',
                style={'textAlign': "center", "padding-bottom": "30"}),
                dcc.Graph(id="cases_groups", 
                          figure = show_first_five_groups())], 
            className="twelve columns"),
    html.H3(id = 'country-name',  children=["init"] , 
            style={'textAlign': "center", "padding-bottom": "30"} ),
    html.Div(dcc.Graph(id="yearly_country"),  id = 'yearly_box', 
             className="five columns"),
    html.Div(dcc.Graph(id="attacks_type"),  id = 'attacks_box', 
             className="five columns"),
    
])

##### Figure Updates with CallBacks ##### 
@app.callback(
    dash.dependencies.Output('country-name', 'children'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_title(clickData):
    text = ''
    if clickData is not None:
        selected_country = clickData['points'][0]['location']
        text = str(selected_country) 
    return text

@app.callback(
    dash.dependencies.Output('yearly_country', 'figure'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_yearly_data(clickData):
    df = ''
    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df = df_death_injury[df_death_injury['id'] == selected_country].sort_values(by='iyear')
        df = df.groupby(by=['iyear']).agg({'nkill':'sum', 'nwound':'sum'}).reset_index().sort_values(by='iyear')
    
    else:
        df = df_death_injury.groupby(by=['iyear']).agg({'nkill':'sum','nwound':'sum'}).reset_index().sort_values(by='iyear')

        
    fig = go.Figure()
    # Create and style traces
    fig.add_trace(go.Scatter(x=df['iyear'].values, 
                             y=df['nkill'].values, 
                             name='# of Killed People',
                             line=dict(color='firebrick', width=4)))
    fig.add_trace(go.Scatter(x=df['iyear'].values, 
                             y=df['nwound'].values, 
                             name = '# of Injured People',
                             line=dict(color='royalblue', width=4)))
    
    fig.update_layout(title_text="Deaths and Injuries(since 2000)",
                      yaxis=dict(showgrid=True),
                     template='simple_white')


    
    return fig

@app.callback(
    dash.dependencies.Output('attacks_type', 'figure'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_target_data(clickData):
    df = ''
    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df = df_attacks_targets[df_attacks_targets['id'] == selected_country]
        
        most_five = df.groupby(by=['targtype1_txt']).agg({'eventid':'sum'}).reset_index().sort_values(by='eventid')['targtype1_txt'].head(10)
        df = df[df['targtype1_txt'].isin(most_five)].groupby(by=['targtype1_txt','iyear']).agg({'eventid':'sum'}).reset_index()

    else:
        most_five = df_attacks_targets.groupby(by=['targtype1_txt']).agg({'eventid':'sum'}).reset_index().sort_values(by='eventid')['targtype1_txt'].head(10)
        df = df_attacks_targets[df_attacks_targets['targtype1_txt'].isin(most_five)].groupby(by=['targtype1_txt','iyear']).agg({'eventid':'sum'}).reset_index()

    fig = px.bar(df, x='iyear', y='eventid', color= 'targtype1_txt')

    fig.update_layout(title_text="Attacks by Target(since 2000)")
    
    return fig  



app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jan/2022 16:28:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:17] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2022 16:28:33] "POST /_dash-update-component HTTP/1.1" 200 -


In [21]:
df_death_injury.groupby(by=['iyear']).agg({'nkill':'sum'}).reset_index()

,iyear,nkill
0,2000,3285
1,2001,3875
2,2002,3783
3,2003,2751
4,2004,5018
5,2005,5261
6,2006,8383
7,2007,11227
8,2008,8351
9,2009,8973
